In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate ftfy spacy

In [ ]:
import subprocess

# The xformers package is mandatory to be able to create several 768x768 images.
github_url = "https://github.com/brian6091/xformers-wheels"
xformer_id = "0.0.15.dev0+4c06c79"
xformers_wheels = f"xformers-{xformer_id}.d20221205-cp38-cp38-linux_x86_64.whl"

# Install xformers using pre-compiled Python wheels
%pip install -q {github_url}/releases/download/{xformer_id}/{xformers_wheels}

In [ ]:
# model_id = "stabilityai/stable-diffusion-2-1-base"
model_id = "stabilityai/stable-diffusion-2-1"

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler

# scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

In [ ]:
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline

device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=torch.float16,
    revision="fp16",
    )
pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

if model_id.endswith('-base'):
  image_length = 512
else:
  image_length = 768



In [ ]:
prompt = "A pikachu fine dining with a view to the Eiffel Tower"
num_images = 4

images = pipe(
    prompt,
    num_images_per_prompt=num_images,
    guidance_scale=9,
    num_inference_steps=25,
    height=image_length,
    width=image_length,
    ).images
    
media.show_images(images)
images[0].save("output.jpg")